In [24]:
# Import libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

In [25]:
# Access calendar
url = "https://www.ccny.cuny.edu/registrar/fall"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

In [26]:
# Extract table data
rows = soup.find_all("tr")
dates = []
dows = []
texts = []

for row in rows:
    cells = row.find_all("td")
    if len(cells) >= 3:
        date = cells[0].text.strip()
        dow = cells[1].text.strip()
        text = cells[2].text.strip()
        
        # Cleaning up discrepancies in "text" column
        text = text.replace("\n", " ").replace("\t", " ").strip()
        # Date ranges
        if "-" in date:
            range = date.split("-")
            startRaw = range[0].strip() + ", 2021"
            month = startRaw.split(" ")[0]
            endRaw = month + " " + range[1].strip() + ", 2021"
            
            # Convert to datetime objects
            startObj = datetime.strptime(startRaw, "%B %d, %Y").date()
            endObj = datetime.strptime(endRaw, "%B %d, %Y").date()

            # Loop through range and append each date
            while startObj <= endObj:
                dates.append(startObj.strftime("%Y-%m-%d"))  # Append formatted date
                dows.append(startObj.strftime("%A"))  # Append correct day of the week
                texts.append(text)
                startObj += timedelta(days=1)

        else:
            # Single dates
            if "2022" in date:
                dateObj = datetime.strptime(date, "%B %d, %Y").date()
            else:
                date += ", 2021"
                dateObj = datetime.strptime(date, "%B %d, %Y").date()

            dates.append(dateObj.strftime("%Y-%m-%d"))
            dows.append(dateObj.strftime("%A"))
            texts.append(text)

In [27]:
# Building DataFrame
calendar = pd.DataFrame({"dow": dows, "text": texts}, index=dates)

# Display DataFrame
print(calendar)

                  dow                                               text
2021-08-01     Sunday  Application for degree for January and Februar...
2021-08-18  Wednesday                 Last day to apply for Study Abroad
2021-08-24    Tuesday  Last day of Registration;    Last day to file ...
2021-08-25  Wednesday  Start of Fall Term;    Classes begin;    Initi...
2021-08-25  Wednesday          Change of program period; late fees apply
2021-08-26   Thursday          Change of program period; late fees apply
2021-08-27     Friday          Change of program period; late fees apply
2021-08-28   Saturday          Change of program period; late fees apply
2021-08-29     Sunday          Change of program period; late fees apply
2021-08-30     Monday          Change of program period; late fees apply
2021-08-31    Tuesday          Change of program period; late fees apply
2021-08-26   Thursday                     Last day for Independent Study
2021-08-28   Saturday                      First da